In [1]:
!pip install -q -U watermark

In [2]:
!pip install -qq transformers

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 3.3MB 38.8MB/s 
     |████████████████████████████████| 901kB 35.5MB/s 


In [3]:
! pip install torch

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import BertModel, BertTokenizer

# specify GPU
device = torch.device("cuda")

In [5]:
!gdown --id 1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
!gdown --id 1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv

Downloading...
From: https://drive.google.com/uc?id=1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
To: /content/apps.csv
100% 134k/134k [00:00<00:00, 34.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv
To: /content/reviews.csv
7.17MB [00:00, 111MB/s]


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_json("/content/drive/MyDrive/Copy of published_jobs_for_job_category_extraction.json")

In [8]:
JOB_CATEGORY_MIN = 280

import pandas as pd

pd.set_option('display.max_rows', None)

dataset = pd.read_json("/content/drive/MyDrive/Copy of published_jobs_for_job_category_extraction.json")

dataset.dropna(axis=0, inplace=True)

dataset_copy = dataset.copy()

counts_job_category = dataset_copy['job_category'].value_counts()

# dataset_after_minimum_category 

df = dataset_copy.loc[dataset_copy['job_category'].isin(counts_job_category.index[counts_job_category > JOB_CATEGORY_MIN])]

print(df.shape)

(42241, 4)


In [9]:
sample_size = 280
df = df.groupby('job_category').apply(lambda x: x.sample(sample_size) if len(x) > 280 else x.sample(len(x)))

In [10]:
df = df.reset_index(level=0, drop=True)

In [11]:
import re
import pandas as pd
import spacy
from bs4 import BeautifulSoup
from spacy.lang.en import STOP_WORDS

# stopwords = stopwords()

In [12]:

nlp = spacy.load('en_core_web_sm')

In [13]:
class EnglishPreprocessor:
    
    def replace_character(self, x_data):
      x_data = x_data.replace({'/': ' ', '&': ' ', ',': ' ', '$': ' ', '\n': ' ', '-':' '}, regex=True)
      return x_data

    def remove_space(self, x_data):
        return x_data.apply(lambda x: " ".join(x.split()))

    def remove_punctuations(self, x_data):
        return x_data.apply(lambda x: re.sub('[^A-Z a-z 0-9]+', '', x))

    def to_lower(self, x_data):
        return x_data.apply(lambda x: x.lower())

    def preprocess(self, x_data):
        # Replace Character
        x_data = self.replace_character(x_data)
        # Remove Punctuation and Special Character
        x_data = self.remove_punctuations(x_data)
        # Remove Multiple Space
        x_data = self.remove_space(x_data)
        # To_lower
        x_data = self.to_lower(x_data)

        return x_data

In [14]:
df = df.iloc[:, 1:3]

In [15]:
x = df['title']
y = df['job_category']

In [16]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [17]:
encoder = LabelEncoder()
y=encoder.fit_transform(y)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [19]:
bert = BertModel.from_pretrained('bert-base-cased')

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [20]:
x_train = EnglishPreprocessor().preprocess(x_train)
x_test = EnglishPreprocessor().preprocess(x_test)

In [21]:
from torch.utils.data import Dataset, DataLoader

In [22]:
class GPReviewDataset(Dataset):

  def __init__(self, title, categories, tokenizer, max_len):
    self.title = title
    self.categories = categories
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.  title)
  
  def __getitem__(self, item):
    title = str(self.title.iloc[item])
    categories = self.categories[item]

    encoding = self.tokenizer.encode_plus(
      title,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'title': title,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'categories': torch.tensor(categories, dtype=torch.long)
    }

In [23]:
def create_data_loader(x,y, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    title=x,
    categories=y,
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [24]:
BATCH_SIZE = 16
MAX_LEN=6

In [25]:
train_data_loader = create_data_loader(x_train,y_train, tokenizer, 6, BATCH_SIZE)
val_data_loader = create_data_loader(x_test,y_test, tokenizer, 6, BATCH_SIZE)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [26]:
data = next(iter(train_data_loader))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a

In [27]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['categories'].shape)

torch.Size([16, 6])
torch.Size([16, 6])
torch.Size([16])


In [29]:
class JobClassifier(nn.Module):

  def __init__(self):
    super(JobClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size,50)
    self.softmax = nn.LogSoftmax()
  
  def forward(self, input_ids, attention_mask):
    bertOutput = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(bertOutput['pooler_output'])
    output = self.softmax(output)
    return self.out(output)

In [30]:
model = JobClassifier()
model = model.to(device)

In [34]:
from transformers import AdamW,get_linear_schedule_with_warmup

In [35]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [41]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["categories"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [42]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["categories"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [43]:

from collections import defaultdict

In [44]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(x_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(x_test)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/10
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a

Train loss 2.5092276760510037 accuracy 0.433125


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest se

Val   loss 1.686838240282876 accuracy 0.5928571428571429

Epoch 2/10
----------


...

Val   loss 1.5666055967126573 accuracy 0.6175

Epoch 10/10
----------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest se

Train loss 0.833595018652933 accuracy 0.7846428571428572


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

Val   loss 1.5717691598619734 accuracy 0.6210714285714286

CPU times: user 13min 35s, sys: 4min 19s, total: 17min 54s
Wall time: 18min 3s


In [ ]:
|